In [9]:
pip install cloudscraper


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import cloudscraper
from bs4 import BeautifulSoup

scraper = cloudscraper.create_scraper()

url = "https://www.albumoftheyear.org/album/508-kanye-west-the-college-dropout.php"
html = scraper.get(url).text

soup = BeautifulSoup(html, "html.parser")
print(soup.select_one("h1.albumTitle span").text)

The College Dropout


In [23]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import os
# === 1. Список всех альбомов ===
album_urls = [
    "https://www.albumoftheyear.org/album/508-kanye-west-the-college-dropout.php",
    "https://www.albumoftheyear.org/album/484-kanye-west-late-registration.php",
    "https://www.albumoftheyear.org/album/435-kanye-west-graduation.php",
    "https://www.albumoftheyear.org/album/844-kanye-west-808s-heartbreak.php",
    "https://www.albumoftheyear.org/album/1998-kanye-west-my-beautiful-dark-twisted-fantasy.php",
    "https://www.albumoftheyear.org/album/2724-jay-z-kanye-west-watch-the-throne.php",
    "https://www.albumoftheyear.org/album/7215-kanye-west-yeezus.php",
    "https://www.albumoftheyear.org/album/48146-kanye-west-the-life-of-pablo.php",
    "https://www.albumoftheyear.org/album/112577-kanye-west-ye.php",
    "https://www.albumoftheyear.org/album/108509-kids-see-ghosts-kids-see-ghosts.php",
    "https://www.albumoftheyear.org/album/173704-kanye-west-jesus-is-king.php",
    "https://www.albumoftheyear.org/album/262792-kanye-west-donda.php",
    "https://www.albumoftheyear.org/album/463133-kanye-west-donda-2.php",
    "https://www.albumoftheyear.org/album/770331-ys-vultures-1.php",
    "https://www.albumoftheyear.org/album/854855-ys-vultures-2.php"
]

# создаём скрапер
scraper = cloudscraper.create_scraper()

all_rows = []

# === 2. Парсер одной страницы ===
def parse_album(url):
    print("Парсим:", url)

    html = scraper.get(url).text
    soup = BeautifulSoup(html, "html.parser")

    # ----- Метаданные альбома -----
    album_title = soup.select_one("h1.albumTitle span").get_text(strip=True)
    artist = soup.select_one(".albumHeadline .artist a").get_text(strip=True)

    release_date = soup.select_one(".albumTopBox.info .detailRow").get_text(strip=True)
    release_date = release_date.replace("/ Release Date", "").strip()

    label_row = soup.select(".albumTopBox.info .detailRow")[2]
    labels = ", ".join(a.get_text(strip=True) for a in label_row.select("a"))

    genre_row = soup.select(".albumTopBox.info .detailRow")[3]
    genres = ", ".join(a.get_text(strip=True) for a in genre_row.select("a"))

    critic_score = soup.select_one(".albumCriticScore span[itemprop='ratingValue']").get_text(strip=True)
    user_score = soup.select_one(".albumUserScore a").get_text(strip=True)

    album_length = soup.select_one(".totalLength")
    if album_length:
        album_length = album_length.get_text(strip=True).replace("Total Length: ", "")
    else:
        album_length = None  # На некоторых альбомах (например Donda 2) может отсутствовать

    # ----- Треки -----
    track_rows = soup.select("table.trackListTable tr")

    for row in track_rows:
        track_number = row.select_one(".trackNumber").get_text(strip=True)
        title = row.select_one(".trackTitle a").get_text(strip=True)
        length = row.select_one(".trackTitle .length").get_text(strip=True)

        feat_block = row.select_one(".featuredArtists")
        featured = ", ".join(a.get_text(strip=True) for a in feat_block.select("a")) if feat_block else None

        rating_el = row.select_one(".trackRating span")
        track_rating = rating_el.get_text(strip=True)
        rating_count = rating_el["title"].replace(" Ratings", "")

        all_rows.append({
            "album_title": album_title,
            "artist": artist,
            "release_date": release_date,
            "labels": labels,
            "genres": genres,
            "critic_score": critic_score,
            "user_score": user_score,
            "album_length": album_length,

            "track_number": track_number,
            "track_title": title,
            "track_length": length,
            "featured_artists": featured,
            "track_rating": track_rating,
            "track_rating_count": rating_count,

            "album_url": url
        })


# === 3. Запуск на всех альбомах ===
for url in album_urls:
    parse_album(url)

# === 4. Конечный датасет ===
df = pd.DataFrame(all_rows)
df.head()

# === 5. Сохранение ===
df.to_csv("kanye_all_albums_tracks.csv", index=False)

print("ГОТОВО! Всего треков собрано:", len(df))


Парсим: https://www.albumoftheyear.org/album/508-kanye-west-the-college-dropout.php
Парсим: https://www.albumoftheyear.org/album/484-kanye-west-late-registration.php
Парсим: https://www.albumoftheyear.org/album/435-kanye-west-graduation.php
Парсим: https://www.albumoftheyear.org/album/844-kanye-west-808s-heartbreak.php
Парсим: https://www.albumoftheyear.org/album/1998-kanye-west-my-beautiful-dark-twisted-fantasy.php
Парсим: https://www.albumoftheyear.org/album/2724-jay-z-kanye-west-watch-the-throne.php
Парсим: https://www.albumoftheyear.org/album/7215-kanye-west-yeezus.php
Парсим: https://www.albumoftheyear.org/album/48146-kanye-west-the-life-of-pablo.php
Парсим: https://www.albumoftheyear.org/album/112577-kanye-west-ye.php
Парсим: https://www.albumoftheyear.org/album/108509-kids-see-ghosts-kids-see-ghosts.php
Парсим: https://www.albumoftheyear.org/album/173704-kanye-west-jesus-is-king.php
Парсим: https://www.albumoftheyear.org/album/262792-kanye-west-donda.php
Парсим: https://www.albu

In [27]:
tracks_data = [
    # === THE COLLEGE DROPOUT (2004) ===
    {"album": "The College Dropout", "year": 2004, "track": "Intro", "main_theme": "social", "subthemes": "education, expectations", "commentary": "Сатира на давление общества и академических норм."},
    {"album": "The College Dropout", "year": 2004, "track": "We Don't Care", "main_theme": "social", "subthemes": "youth, poverty, inequality", "commentary": "Критика городской бедности и отсутствия возможностей."},
    {"album": "The College Dropout", "year": 2004, "track": "Graduation Day", "main_theme": "social", "subthemes": "discipline, conformity", "commentary": "Призыв не подчиняться системе образования."},
    {"album": "The College Dropout", "year": 2004, "track": "All Falls Down", "main_theme": "identity", "subthemes": "insecurity, capitalism", "commentary": "О низкой самооценке и потребительских стандартах."},
    {"album": "The College Dropout", "year": 2004, "track": "I'll Fly Away", "main_theme": "religion", "subthemes": "freedom", "commentary": "Госпел-вставка, символ духовного освобождения."},
    {"album": "The College Dropout", "year": 2004, "track": "Spaceship", "main_theme": "work", "subthemes": "exploitation, dreams", "commentary": "Про рутинную работу в магазине и мечту о выходе из цикла."},
    {"album": "The College Dropout", "year": 2004, "track": "Jesus Walks", "main_theme": "religion", "subthemes": "faith, salvation", "commentary": "Революционный трек о христианстве в хип-хопе."},
    {"album": "The College Dropout", "year": 2004, "track": "Never Let Me Down", "main_theme": "motivation", "subthemes": "race, inspiration", "commentary": "О стойкости и наследии в борьбе с несправедливостью."},
    {"album": "The College Dropout", "year": 2004, "track": "Get Em High", "main_theme": "lifestyle", "subthemes": "ego, weed", "commentary": "Бравирующий трек о статусе и кайфе."},
    {"album": "The College Dropout", "year": 2004, "track": "Workout Plan", "main_theme": "humor", "subthemes": "beauty standards", "commentary": "Пародия на фитнес-индустрию."},
    {"album": "The College Dropout", "year": 2004, "track": "The New Workout Plan", "main_theme": "satire", "subthemes": "body image", "commentary": "Ирония над культами красоты."},
    {"album": "The College Dropout", "year": 2004, "track": "Slow Jamz", "main_theme": "romance", "subthemes": "humor", "commentary": "Игривая любовная композиция."},
    {"album": "The College Dropout", "year": 2004, "track": "Breathe In Breathe Out", "main_theme": "ego", "subthemes": "rap identity", "commentary": "Классический соревновательный рэп."},
    {"album": "The College Dropout", "year": 2004, "track": "School Spirit Skit 1", "main_theme": "satire", "subthemes": "education", "commentary": "Насмешка над университетскими правилами."},
    {"album": "The College Dropout", "year": 2004, "track": "School Spirit", "main_theme": "rebellion", "subthemes": "youth, identity", "commentary": "Бунт против академической культуры."},
    {"album": "The College Dropout", "year": 2004, "track": "School Spirit Skit 2", "main_theme": "satire", "subthemes": "conformity", "commentary": "Продолжение комической критики образования."},
    {"album": "The College Dropout", "year": 2004, "track": "Lil Jimmy Skit", "main_theme": "satire", "subthemes": "family expectations", "commentary": "О том, как семьи навязывают детям путь учёбы."},
    {"album": "The College Dropout", "year": 2004, "track": "Two Words", "main_theme": "social", "subthemes": "race, aggression", "commentary": "Сильный протестный рэп о расовой реальности."},
    {"album": "The College Dropout", "year": 2004, "track": "Through the Wire", "main_theme": "trauma", "subthemes": "survival, recovery", "commentary": "Трек, записанный после аварии — символ стойкости."},
    {"album": "The College Dropout", "year": 2004, "track": "Family Business", "main_theme": "family", "subthemes": "nostalgia, roots", "commentary": "Очень тёплая история о семейных ценностях."},
    {"album": "The College Dropout", "year": 2004, "track": "Last Call", "main_theme": "legacy", "subthemes": "career, ambition", "commentary": "Автобиографический манифест о пути Канье."},

    # === LATE REGISTRATION (2005) ===
    {"album": "Late Registration", "year": 2005, "track": "Wake Up Mr. West", "main_theme": "social", "subthemes": "fame, criticism", "commentary": "Индустрия требует от артиста всё больше."},
    {"album": "Late Registration", "year": 2005, "track": "Heard 'Em Say", "main_theme": "social", "subthemes": "hope, struggle", "commentary": "О трудностях жизни и поисках надежды."},
    {"album": "Late Registration", "year": 2005, "track": "Touch the Sky", "main_theme": "ambition", "subthemes": "success, risk", "commentary": "Гимн стремления к вершинам."},
    {"album": "Late Registration", "year": 2005, "track": "Gold Digger", "main_theme": "relationships", "subthemes": "money, conflict", "commentary": "Ирония на финансовые отношения между полами."},
    {"album": "Late Registration", "year": 2005, "track": "Skit #1", "main_theme": "satire", "subthemes": "fraternity", "commentary": "Пародия на студенческие братства."},
    {"album": "Late Registration", "year": 2005, "track": "Drive Slow", "main_theme": "wisdom", "subthemes": "life lessons", "commentary": "Призыв не спешить в погоне за славой."},
    {"album": "Late Registration", "year": 2005, "track": "My Way Home", "main_theme": "identity", "subthemes": "race", "commentary": "Размышления об афроамериканской реальности."},
    {"album": "Late Registration", "year": 2005, "track": "Crack Music", "main_theme": "politics", "subthemes": "systemic oppression", "commentary": "Сильная политическая критика Америки."},
    {"album": "Late Registration", "year": 2005, "track": "Roses", "main_theme": "family", "subthemes": "illness, grief", "commentary": "О болезни родственницы и беспомощности."},
    {"album": "Late Registration", "year": 2005, "track": "Bring Me Down", "main_theme": "ego", "subthemes": "haters", "commentary": "Ответ критикам и недоброжелателям."},
    {"album": "Late Registration", "year": 2005, "track": "Addiction", "main_theme": "introspection", "subthemes": "temptation", "commentary": "О борьбе с искушениями."},
    {"album": "Late Registration", "year": 2005, "track": "Skit #2", "main_theme": "satire", "subthemes": "capitalism", "commentary": "Насмешка над финансовыми ловушками."},
    {"album": "Late Registration", "year": 2005, "track": "Diamonds from Sierra Leone", "main_theme": "politics", "subthemes": "global exploitation", "commentary": "О цене алмазной индустрии."},
    {"album": "Late Registration", "year": 2005, "track": "We Major", "main_theme": "celebration", "subthemes": "success", "commentary": "Празднование карьерного взлёта."},
    {"album": "Late Registration", "year": 2005, "track": "Skit #3", "main_theme": "satire", "subthemes": "poverty", "commentary": "Ироничный социальный комментарий."},
    {"album": "Late Registration", "year": 2005, "track": "Hey Mama", "main_theme": "family", "subthemes": "gratitude, motherhood", "commentary": "Самый личный трек посвящённый матери."},
    {"album": "Late Registration", "year": 2005, "track": "Celebration", "main_theme": "lifestyle", "subthemes": "party, fame", "commentary": "Лёгкая композиция о наслаждении жизнью."},
    {"album": "Late Registration", "year": 2005, "track": "Skit #4", "main_theme": "satire", "subthemes": "financial pressure", "commentary": "Ещё один удар по экономическим реалиям."},
    {"album": "Late Registration", "year": 2005, "track": "Gone", "main_theme": "career", "subthemes": "transition", "commentary": "О переменах в жизни артиста."},
    {"album": "Late Registration", "year": 2005, "track": "Diamonds Remix", "main_theme": "politics", "subthemes": "wealth inequality", "commentary": "Расширенная политическая критика."},
    {"album": "Late Registration", "year": 2005, "track": "Late", "main_theme": "introspection", "subthemes": "self-worth", "commentary": "Завершающий взгляд на своё место в мире."},

    # === GRADUATION (начало) ===
    {"album": "Graduation", "year": 2007, "track": "Good Morning", "main_theme": "motivation", "subthemes": "self-growth", "commentary": "Открытие новой главы жизни."},
    {"album": "Graduation", "year": 2007, "track": "Champion", "main_theme": "self-worth", "subthemes": "confidence", "commentary": "Утверждение собственной ценности."},
    {"album": "Graduation", "year": 2007, "track": "Stronger", "main_theme": "ambition", "subthemes": "resilience", "commentary": "Становление сильнее через боль."},
    {"album": "Graduation", "year": 2007, "track": "I Wonder", "main_theme": "introspection", "subthemes": "destiny", "commentary": "Размышления о поиске себя."},
    {"album": "Graduation", "year": 2007, "track": "Good Life", "main_theme": "celebration", "subthemes": "fame", "commentary": "Эйфория успеха."},
    {"album": "Graduation", "year": 2007, "track": "Can't Tell Me Nothing", "main_theme": "rebellion", "subthemes": "ego", "commentary": "Одна из вершин его эго-периода."},
    {"album": "Graduation", "year": 2007, "track": "Barry Bonds", "main_theme": "ego", "subthemes": "competition", "commentary": "Соревнование и давление славы."},
    {"album": "Graduation", "year": 2007, "track": "Drunk and Hot Girls", "main_theme": "emptiness", "subthemes": "nightlife", "commentary": "Критика бессмысленности клубной жизни."},
    {"album": "Graduation", "year": 2007, "track": "Flashing Lights", "main_theme": "relationships", "subthemes": "fame, distance", "commentary": "О напряжённых отношениях в публичности."},
    {"album": "Graduation", "year": 2007, "track": "Everything I Am", "main_theme": "introspection", "subthemes": "self-acceptance", "commentary": "Принятие своей уникальности."},
    {"album": "Graduation", "year": 2007, "track": "The Glory", "main_theme": "ambition", "subthemes": "legacy", "commentary": "Утверждение величия."},
    {"album": "Graduation", "year": 2007, "track": "Homecoming", "main_theme": "nostalgia", "subthemes": "identity, roots", "commentary": "О связи Канье с Чикаго."},
    {"album": "Graduation", "year": 2007, "track": "Big Brother", "main_theme": "relationships", "subthemes": "mentorship, conflict", "commentary": "Посвящается сложным отношениям с Jay-Z."},
    # === 808s & HEARTBREAK (2008) ===
    {"album": "808s & Heartbreak", "year": 2008, "track": "Say You Will", "main_theme": "heartbreak", "subthemes": "longing, emptiness", "commentary": "Минималистичная боль расставания и ожидания."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "Welcome to Heartbreak", "main_theme": "emotion", "subthemes": "envy, regret", "commentary": "Зависть к 'нормальной жизни', одиночество среди успеха."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "Heartless", "main_theme": "heartbreak", "subthemes": "betrayal", "commentary": "Токсичные отношения и эмоциональная холодность."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "Amazing", "main_theme": "ego", "subthemes": "resilience", "commentary": "Смесь мании величия и самоутверждения."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "Love Lockdown", "main_theme": "emotion", "subthemes": "isolation", "commentary": "Замкнутость и попытка подавить чувства."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "Paranoid", "main_theme": "emotion", "subthemes": "fear, anxiety", "commentary": "Паника, тревога и недоверие."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "RoboCop", "main_theme": "relationships", "subthemes": "control, conflict", "commentary": "Отношения как механизм контроля."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "Street Lights", "main_theme": "introspection", "subthemes": "life passing, melancholy", "commentary": "Медитация о времени и одиночестве."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "Bad News", "main_theme": "heartbreak", "subthemes": "shock, fear", "commentary": "Неизбежность эмоциональной боли."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "See You in My Nightmares", "main_theme": "anger", "subthemes": "rejection", "commentary": "Разрыв, где гнев доминирует над печалью."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "Coldest Winter", "main_theme": "grief", "subthemes": "loss", "commentary": "Смерть матери, самый эмоционально тяжёлый трек альбома."},
    {"album": "808s & Heartbreak", "year": 2008, "track": "Pinocchio Story (Live)", "main_theme": "identity", "subthemes": "fame, emptiness", "commentary": "Желание снова стать настоящим, искренним."},

    # === MY BEAUTIFUL DARK TWISTED FANTASY (2010) ===
    {"album": "MBDTF", "year": 2010, "track": "Dark Fantasy", "main_theme": "ego", "subthemes": "fame, surrealism", "commentary": "Фантазия о величии, вступление в тёмный мир славы."},
    {"album": "MBDTF", "year": 2010, "track": "Gorgeous", "main_theme": "social", "subthemes": "race, frustration", "commentary": "О расизме и политической реальности."},
    {"album": "MBDTF", "year": 2010, "track": "POWER", "main_theme": "ego", "subthemes": "self-destruction", "commentary": "Гимн гипер-величия и, одновременно, краха."},
    {"album": "MBDTF", "year": 2010, "track": "All of the Lights", "main_theme": "social", "subthemes": "abuse, chaos", "commentary": "История семейного кризиса и последствий славы."},
    {"album": "MBDTF", "year": 2010, "track": "Monster", "main_theme": "ego", "subthemes": "fame, duality", "commentary": "Монструозность славы и внутренние демоны."},
    {"album": "MBDTF", "year": 2010, "track": "So Appalled", "main_theme": "industry", "subthemes": "fame, decadence", "commentary": "Пустота богатства и абсурд индустрии."},
    {"album": "MBDTF", "year": 2010, "track": "Devil in a New Dress", "main_theme": "romance", "subthemes": "temptation, betrayal", "commentary": "Романтизированная, но опасная любовь."},
    {"album": "MBDTF", "year": 2010, "track": "Runaway", "main_theme": "introspection", "subthemes": "regret, toxicity", "commentary": "Извинение перед бывшими, признание собственной вины."},
    {"album": "MBDTF", "year": 2010, "track": "Hell Of A Life", "main_theme": "hedonism", "subthemes": "porn, excess", "commentary": "Излишество, падение и наркотическая слава."},
    {"album": "MBDTF", "year": 2010, "track": "Blame Game", "main_theme": "relationships", "subthemes": "fights, breakup", "commentary": "Токсичные отношения и обвинения."},
    {"album": "MBDTF", "year": 2010, "track": "Lost in the World", "main_theme": "identity", "subthemes": "love, confusion", "commentary": "Поиск себя через любовь."},
    {"album": "MBDTF", "year": 2010, "track": "Who Will Survive in America?", "main_theme": "politics", "subthemes": "race, justice", "commentary": "Социальная критика Америки."},

    # === WATCH THE THRONE (2011) ===
    {"album": "Watch the Throne", "year": 2011, "track": "No Church in the Wild", "main_theme": "philosophy", "subthemes": "morality, rebellion", "commentary": "Вопросы морали, власти, бунта."},
    {"album": "Watch the Throne", "year": 2011, "track": "Lift Off", "main_theme": "ambition", "subthemes": "success", "commentary": "Полет к вершинам славы."},
    {"album": "Watch the Throne", "year": 2011, "track": "Niggas in Paris", "main_theme": "celebration", "subthemes": "luxury, ego", "commentary": "Эйфория роскошной жизни."},
    {"album": "Watch the Throne", "year": 2011, "track": "Otis", "main_theme": "ego", "subthemes": "rap excellence", "commentary": "Праздник мастерства рэпа."},
    {"album": "Watch the Throne", "year": 2011, "track": "Gotta Have It", "main_theme": "ambition", "subthemes": "success", "commentary": "Жажда новых достижений."},
    {"album": "Watch the Throne", "year": 2011, "track": "New Day", "main_theme": "family", "subthemes": "fatherhood, legacy", "commentary": "Послание будущим детям."},
    {"album": "Watch the Throne", "year": 2011, "track": "That's My Bitch", "main_theme": "race", "subthemes": "beauty standards", "commentary": "О чернокожей женственности и западных стандартах."},
    {"album": "Watch the Throne", "year": 2011, "track": "Welcome to the Jungle", "main_theme": "trauma", "subthemes": "stress, chaos", "commentary": "Жизнь в хаосе славы."},
    {"album": "Watch the Throne", "year": 2011, "track": "Who Gon Stop Me", "main_theme": "ego", "subthemes": "fearlessness", "commentary": "Гимн непобедимости."},
    {"album": "Watch the Throne", "year": 2011, "track": "Murder to Excellence", "main_theme": "social", "subthemes": "violence, race", "commentary": "О цикле насилия и успехе чернокожих артистов."},
    {"album": "Watch the Throne", "year": 2011, "track": "Made in America", "main_theme": "religion", "subthemes": "gratitude", "commentary": "Благодарность Богу и предкам."},
    {"album": "Watch the Throne", "year": 2011, "track": "Why I Love You", "main_theme": "relationships", "subthemes": "betrayal", "commentary": "Потеря друзей и предательство."},

    # === YEEZUS (2013) — часть ===
    {"album": "Yeezus", "year": 2013, "track": "On Sight", "main_theme": "chaos", "subthemes": "shock, industrial", "commentary": "Агрессивный манифест нового звучания."},
    {"album": "Yeezus", "year": 2013, "track": "Black Skinhead", "main_theme": "social", "subthemes": "race, rage", "commentary": "Средний палец расизму и культурному угнетению."},
    {"album": "Yeezus", "year": 2013, "track": "I Am a God", "main_theme": "ego", "subthemes": "divinity, mania", "commentary": "Гиперболизированное величие и мания."},
    {"album": "Yeezus", "year": 2013, "track": "New Slaves", "main_theme": "politics", "subthemes": "consumerism, racism", "commentary": "О современном рабстве брендов и систем."},
    {"album": "Yeezus", "year": 2013, "track": "Hold My Liquor", "main_theme": "emotion", "subthemes": "self-destruction", "commentary": "Падение и сквозная депрессия."}
]
tracks_data += [
    # === YEEZUS (2013) — продолжение ===
    {"album": "Yeezus", "year": 2013, "track": "I'm In It", "main_theme": "sexuality", "subthemes": "chaos, obsession", "commentary": "Грубая, хаотичная сексуальность как часть образа Yeezus."},
    {"album": "Yeezus", "year": 2013, "track": "Blood on the Leaves", "main_theme": "social", "subthemes": "race, tragedy", "commentary": "Переплетение личной драмы и истории о расовом насилии."},
    {"album": "Yeezus", "year": 2013, "track": "Guilt Trip", "main_theme": "emotion", "subthemes": "regret, heartbreak", "commentary": "Чувство вины и эмоциональная неустойчивость."},
    {"album": "Yeezus", "year": 2013, "track": "Send It Up", "main_theme": "nightlife", "subthemes": "chaos", "commentary": "Клубная агрессия и постмодерн."},
    {"album": "Yeezus", "year": 2013, "track": "Bound 2", "main_theme": "love", "subthemes": "nostalgia, family", "commentary": "Тёплый ретро-трек, контрастирующий с остальным альбомом."},

    # === THE LIFE OF PABLO (2016) ===
    {"album": "The Life of Pablo", "year": 2016, "track": "Ultralight Beam", "main_theme": "religion", "subthemes": "salvation, hope", "commentary": "Молитвенный гимн, начало новой духовной эры."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Father Stretch My Hands Pt. 1", "main_theme": "emotion", "subthemes": "family, redemption", "commentary": "Песня о внутреннем разрыве — между верой и хаосом."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Pt. 2", "main_theme": "chaos", "subthemes": "youth, trauma", "commentary": "Сюрреалистичное продолжение, контраст мира и разрушения."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Famous", "main_theme": "fame", "subthemes": "ego, controversy", "commentary": "Скандальный трек о знаменитости и медиа-войнах."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Feedback", "main_theme": "ego", "subthemes": "identity", "commentary": "Грубое хвастовство и самоирония."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Low Lights", "main_theme": "religion", "subthemes": "testimony", "commentary": "Госпел-монолог о Божьем водительстве."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Highlights", "main_theme": "celebration", "subthemes": "ego", "commentary": "Праздничное перечисление достижений."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Freestyle 4", "main_theme": "chaos", "subthemes": "dark sexuality", "commentary": "Беспокойное, инстинктивное выражение тьмы."},
    {"album": "The Life of Pablo", "year": 2016, "track": "I Love Kanye", "main_theme": "meta", "subthemes": "self-parody", "commentary": "Самоироничное обращение к фанатам."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Waves", "main_theme": "emotion", "subthemes": "renewal", "commentary": "О преодолении боли и переменах."},
    {"album": "The Life of Pablo", "year": 2016, "track": "FML", "main_theme": "relationships", "subthemes": "fear, vulnerability", "commentary": "Раскрытие личных страхов и саморазрушения."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Real Friends", "main_theme": "relationships", "subthemes": "trust, betrayal", "commentary": "О сложности доверия в окружении."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Wolves", "main_theme": "emotion", "subthemes": "fear, insecurity", "commentary": "О чувстве беззащитности и потерянности."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Frank's Track", "main_theme": "introspection", "subthemes": "emptiness", "commentary": "Минималистичное духовное размышление."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Siiiiiiiiilver Surffffeeeeer Intermission", "main_theme": "friendship", "subthemes": "loyalty", "commentary": "Сообщение о личной поддержке."},
    {"album": "The Life of Pablo", "year": 2016, "track": "30 Hours", "main_theme": "nostalgia", "subthemes": "memory", "commentary": "О старых отношениях и жизненных периодах."},
    {"album": "The Life of Pablo", "year": 2016, "track": "No More Parties in LA", "main_theme": "social", "subthemes": "Hollywood toxicity", "commentary": "Критика индустрии развлечений."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Facts", "main_theme": "business", "subthemes": "Nike, Adidas", "commentary": "Дисс на Nike и заявление силы Yeezy."},
    {"album": "The Life of Pablo", "year": 2016, "track": "Fade", "main_theme": "relationships", "subthemes": "fear of loss", "commentary": "Об исчезновении чувств и близости."},

    # === YE (2018) ===
    {"album": "Ye", "year": 2018, "track": "I Thought About Killing You", "main_theme": "mental_health", "subthemes": "dark thoughts, self-awareness", "commentary": "Исповедь о борьбе с ментальными демонами."},
    {"album": "Ye", "year": 2018, "track": "Yikes", "main_theme": "mental_health", "subthemes": "mania, bipolar", "commentary": "Биполярность как суперсила и проклятие."},
    {"album": "Ye", "year": 2018, "track": "All Mine", "main_theme": "relationships", "subthemes": "sex, ego", "commentary": "Размышление о желаниях и верности."},
    {"album": "Ye", "year": 2018, "track": "Wouldn't Leave", "main_theme": "family", "subthemes": "marriage, loyalty", "commentary": "Исповедь о поддержке жены во время скандалов."},
    {"album": "Ye", "year": 2018, "track": "No Mistakes", "main_theme": "forgiveness", "subthemes": "renewal", "commentary": "Примирение и принятие себя."},
    {"album": "Ye", "year": 2018, "track": "Ghost Town", "main_theme": "freedom", "subthemes": "pain, release", "commentary": "Освобождение от боли и старых форм."},
    {"album": "Ye", "year": 2018, "track": "Violent Crimes", "main_theme": "family", "subthemes": "fatherhood, fear", "commentary": "Страх за своих дочерей, зрелость."},

    # === KIDS SEE GHOSTS (2018) ===
    {"album": "Kids See Ghosts", "year": 2018, "track": "Feel the Love", "main_theme": "emotion", "subthemes": "rage, catharsis", "commentary": "Взрывная эмоциональная разрядка."},
    {"album": "Kids See Ghosts", "year": 2018, "track": "Fire", "main_theme": "healing", "subthemes": "accountability", "commentary": "Признание прошлых ошибок."},
    {"album": "Kids See Ghosts", "year": 2018, "track": "4th Dimension", "main_theme": "surrealism", "subthemes": "chaos", "commentary": "Шизофреническое смешение эпох и звуков."},
    {"album": "Kids See Ghosts", "year": 2018, "track": "Freeee (Ghost Town Pt. 2)", "main_theme": "freedom", "subthemes": "mental liberation", "commentary": "Обретение свободы от внутренних цепей."},
    {"album": "Kids See Ghosts", "year": 2018, "track": "Reborn", "main_theme": "healing", "subthemes": "self-growth", "commentary": "Новая жизнь после страданий."},
    {"album": "Kids See Ghosts", "year": 2018, "track": "Kids See Ghosts", "main_theme": "introspection", "subthemes": "fear, courage", "commentary": "Прошлое преследует, но укрепляет."},
    {"album": "Kids See Ghosts", "year": 2018, "track": "Cudi Montage", "main_theme": "redemption", "subthemes": "struggle, faith", "commentary": "О борьбе с зависимостями, молитвенное настроение."},
]
tracks_data += [
    # === JESUS IS KING (2019) ===
    {"album": "Jesus Is King", "year": 2019, "track": "Every Hour", "main_theme": "religion", "subthemes": "worship, purity", "commentary": "Госпел-хорал, полный посвящения Богу."},
    {"album": "Jesus Is King", "year": 2019, "track": "Selah", "main_theme": "religion", "subthemes": "scripture, power", "commentary": "Библейские аллюзии, проповеднический стиль."},
    {"album": "Jesus Is King", "year": 2019, "track": "Follow God", "main_theme": "religion", "subthemes": "obedience, conflict", "commentary": "Борьба за праведный путь, внутренние разрывы."},
    {"album": "Jesus Is King", "year": 2019, "track": "Closed on Sunday", "main_theme": "family", "subthemes": "religion, discipline", "commentary": "Семейные ценности + религиозная строгость."},
    {"album": "Jesus Is King", "year": 2019, "track": "On God", "main_theme": "religion", "subthemes": "gratitude, testimony", "commentary": "Благодарность Богу за спасение и успех."},
    {"album": "Jesus Is King", "year": 2019, "track": "Everything We Need", "main_theme": "faith", "subthemes": "acceptance", "commentary": "Принятие Божьей воли."},
    {"album": "Jesus Is King", "year": 2019, "track": "Water", "main_theme": "purity", "subthemes": "renewal", "commentary": "Очищение и перерождение."},
    {"album": "Jesus Is King", "year": 2019, "track": "God Is", "main_theme": "worship", "subthemes": "devotion", "commentary": "Полностью молитвенное посвящение Богу."},
    {"album": "Jesus Is King", "year": 2019, "track": "Hands On", "main_theme": "religion", "subthemes": "judgment, persecution", "commentary": "Критика церкви, не верящей в его искренность."},
    {"album": "Jesus Is King", "year": 2019, "track": "Use This Gospel", "main_theme": "faith", "subthemes": "redemption", "commentary": "Евангелие как оружие против зла."},
    {"album": "Jesus Is King", "year": 2019, "track": "Jesus Is Lord", "main_theme": "religion", "subthemes": "praise", "commentary": "Короткая декларация веры."},

    # === DONDA (2021) ===
    {"album": "Donda", "year": 2021, "track": "Donda Chant", "main_theme": "grief", "subthemes": "mother, memory", "commentary": "Медитативное повторение имени матери."},
    {"album": "Donda", "year": 2021, "track": "Jail", "main_theme": "conflict", "subthemes": "freedom, mistakes", "commentary": "Осуждение, искупление и токсичные связи."},
    {"album": "Donda", "year": 2021, "track": "God Breathed", "main_theme": "faith", "subthemes": "rebirth", "commentary": "Символ дыхания Бога, возрождение духа."},
    {"album": "Donda", "year": 2021, "track": "Off the Grid", "main_theme": "rebellion", "subthemes": "identity", "commentary": "Энергия отрыва от системы, кристаллизированное эго."},
    {"album": "Donda", "year": 2021, "track": "Hurricane", "main_theme": "redemption", "subthemes": "struggle, salvation", "commentary": "Борьба грешника с искушениями."},
    {"album": "Donda", "year": 2021, "track": "Praise God", "main_theme": "religion", "subthemes": "testimony", "commentary": "Духовное свидетельство и религиозный импульс."},
    {"album": "Donda", "year": 2021, "track": "Jonah", "main_theme": "trauma", "subthemes": "abandonment", "commentary": "О предательстве, боли и одиночестве."},
    {"album": "Donda", "year": 2021, "track": "Ok Ok", "main_theme": "conflict", "subthemes": "betrayal", "commentary": "Ответ врагам и неверности окружения."},
    {"album": "Donda", "year": 2021, "track": "Junya", "main_theme": "ego", "subthemes": "fashion, flex", "commentary": "Хвастовство статусом и стилем."},
    {"album": "Donda", "year": 2021, "track": "Believe What I Say", "main_theme": "relationships", "subthemes": "trust issues", "commentary": "Конфликт доверия и разочарования."},
    {"album": "Donda", "year": 2021, "track": "24", "main_theme": "religion", "subthemes": "hope, worship", "commentary": "Просьба к Богу о спасении, посвящение Коби Брайанту."},
    {"album": "Donda", "year": 2021, "track": "Remote Control", "main_theme": "ego", "subthemes": "power", "commentary": "Контроль и власть над судьбой."},
    {"album": "Donda", "year": 2021, "track": "Moon", "main_theme": "emotion", "subthemes": "escape", "commentary": "Поиск покоя в другом мире."},
    {"album": "Donda", "year": 2021, "track": "Heaven and Hell", "main_theme": "religion", "subthemes": "judgment", "commentary": "Дихотомия добра и зла."},
    {"album": "Donda", "year": 2021, "track": "Donda", "main_theme": "family", "subthemes": "mother", "commentary": "Память о матери, связь поколений."},
    {"album": "Donda", "year": 2021, "track": "Keep My Spirit Alive", "main_theme": "faith", "subthemes": "struggle", "commentary": "Духовная стойкость в трудностях."},
    {"album": "Donda", "year": 2021, "track": "Jesus Lord", "main_theme": "religion", "subthemes": "suffering, redemption", "commentary": "Один из самых серьёзных повествований о боли и спасении."},
    {"album": "Donda", "year": 2021, "track": "New Again", "main_theme": "forgiveness", "subthemes": "rebirth", "commentary": "Очистительное обновление после грехов."},
    {"album": "Donda", "year": 2021, "track": "Tell the Vision", "main_theme": "tribute", "subthemes": "Pop Smoke", "commentary": "Посвящение покойному Pop Smoke."},
    {"album": "Donda", "year": 2021, "track": "Lord I Need You", "main_theme": "relationships", "subthemes": "family, forgiveness", "commentary": "Молитва о сохранении семьи, посвящение Ким."},
    {"album": "Donda", "year": 2021, "track": "Pure Souls", "main_theme": "healing", "subthemes": "optimism", "commentary": "Трек о духовной лёгкости и новом начале."},
    {"album": "Donda", "year": 2021, "track": "Come to Life", "main_theme": "redemption", "subthemes": "emotion, transcendence", "commentary": "Оживление души, центральный момент альбома."},
    {"album": "Donda", "year": 2021, "track": "No Child Left Behind", "main_theme": "faith", "subthemes": "spiritual message", "commentary": "Утверждение Божьей поддержки."},

    # === DONDA 2 (2022) — часть ===
    {"album": "Donda 2", "year": 2022, "track": "True Love", "main_theme": "family", "subthemes": "divorce, children", "commentary": "Боль расставания и отношения с детьми."},
    {"album": "Donda 2", "year": 2022, "track": "Broken Road", "main_theme": "emotion", "subthemes": "reflection", "commentary": "Тёмные размышления о судьбе."},
    {"album": "Donda 2", "year": 2022, "track": "Get Lost", "main_theme": "introspection", "subthemes": "emptiness", "commentary": "Этюд о потерянной личности."},
    {"album": "Donda 2", "year": 2022, "track": "Too Easy", "main_theme": "ego", "subthemes": "success", "commentary": "Шоу-оф и самоутверждение."},
    {"album": "Donda 2", "year": 2022, "track": "Security", "main_theme": "anger", "subthemes": "paranoia, conflict", "commentary": "Паранойя и защита личных границ."},
    {"album": "Donda 2", "year": 2022, "track": "We Did It Kid", "main_theme": "celebration", "subthemes": "legacy", "commentary": "Победа, успех, достижение мечты."},
    {"album": "Donda 2", "year": 2022, "track": "Pablo", "main_theme": "identity", "subthemes": "chaos", "commentary": "Вихрь образов, нарциссизм."},
    {"album": "Donda 2", "year": 2022, "track": "Broken Road", "main_theme": "emotion", "subthemes": "reflection", "commentary": "Путь боли и исцеления."},
    {"album": "Donda 2", "year": 2022, "track": "I'm Finna Love Me", "main_theme": "self-love", "subthemes": "healing", "commentary": "О любви к себе и независимости."},
    {"album": "Donda 2", "year": 2022, "track": "Sci-Fi", "main_theme": "chaos", "subthemes": "divorce, surrealism", "commentary": "Разрыв отношений через футуристический образ."},
]
tracks_data += [
    # === JESUS IS KING (2019) ===
    {"album": "Jesus Is King", "year": 2019, "track": "Every Hour", "main_theme": "religion", "subthemes": "worship, purity", "commentary": "Госпел-хорал, полный посвящения Богу."},
    {"album": "Jesus Is King", "year": 2019, "track": "Selah", "main_theme": "religion", "subthemes": "scripture, power", "commentary": "Библейские аллюзии, проповеднический стиль."},
    {"album": "Jesus Is King", "year": 2019, "track": "Follow God", "main_theme": "religion", "subthemes": "obedience, conflict", "commentary": "Борьба за праведный путь, внутренние разрывы."},
    {"album": "Jesus Is King", "year": 2019, "track": "Closed on Sunday", "main_theme": "family", "subthemes": "religion, discipline", "commentary": "Семейные ценности + религиозная строгость."},
    {"album": "Jesus Is King", "year": 2019, "track": "On God", "main_theme": "religion", "subthemes": "gratitude, testimony", "commentary": "Благодарность Богу за спасение и успех."},
    {"album": "Jesus Is King", "year": 2019, "track": "Everything We Need", "main_theme": "faith", "subthemes": "acceptance", "commentary": "Принятие Божьей воли."},
    {"album": "Jesus Is King", "year": 2019, "track": "Water", "main_theme": "purity", "subthemes": "renewal", "commentary": "Очищение и перерождение."},
    {"album": "Jesus Is King", "year": 2019, "track": "God Is", "main_theme": "worship", "subthemes": "devotion", "commentary": "Полностью молитвенное посвящение Богу."},
    {"album": "Jesus Is King", "year": 2019, "track": "Hands On", "main_theme": "religion", "subthemes": "judgment, persecution", "commentary": "Критика церкви, не верящей в его искренность."},
    {"album": "Jesus Is King", "year": 2019, "track": "Use This Gospel", "main_theme": "faith", "subthemes": "redemption", "commentary": "Евангелие как оружие против зла."},
    {"album": "Jesus Is King", "year": 2019, "track": "Jesus Is Lord", "main_theme": "religion", "subthemes": "praise", "commentary": "Короткая декларация веры."},

    # === DONDA (2021) ===
    {"album": "Donda", "year": 2021, "track": "Donda Chant", "main_theme": "grief", "subthemes": "mother, memory", "commentary": "Медитативное повторение имени матери."},
    {"album": "Donda", "year": 2021, "track": "Jail", "main_theme": "conflict", "subthemes": "freedom, mistakes", "commentary": "Осуждение, искупление и токсичные связи."},
    {"album": "Donda", "year": 2021, "track": "God Breathed", "main_theme": "faith", "subthemes": "rebirth", "commentary": "Символ дыхания Бога, возрождение духа."},
    {"album": "Donda", "year": 2021, "track": "Off the Grid", "main_theme": "rebellion", "subthemes": "identity", "commentary": "Энергия отрыва от системы, кристаллизированное эго."},
    {"album": "Donda", "year": 2021, "track": "Hurricane", "main_theme": "redemption", "subthemes": "struggle, salvation", "commentary": "Борьба грешника с искушениями."},
    {"album": "Donda", "year": 2021, "track": "Praise God", "main_theme": "religion", "subthemes": "testimony", "commentary": "Духовное свидетельство и религиозный импульс."},
    {"album": "Donda", "year": 2021, "track": "Jonah", "main_theme": "trauma", "subthemes": "abandonment", "commentary": "О предательстве, боли и одиночестве."},
    {"album": "Donda", "year": 2021, "track": "Ok Ok", "main_theme": "conflict", "subthemes": "betrayal", "commentary": "Ответ врагам и неверности окружения."},
    {"album": "Donda", "year": 2021, "track": "Junya", "main_theme": "ego", "subthemes": "fashion, flex", "commentary": "Хвастовство статусом и стилем."},
    {"album": "Donda", "year": 2021, "track": "Believe What I Say", "main_theme": "relationships", "subthemes": "trust issues", "commentary": "Конфликт доверия и разочарования."},
    {"album": "Donda", "year": 2021, "track": "24", "main_theme": "religion", "subthemes": "hope, worship", "commentary": "Просьба к Богу о спасении, посвящение Коби Брайанту."},
    {"album": "Donda", "year": 2021, "track": "Remote Control", "main_theme": "ego", "subthemes": "power", "commentary": "Контроль и власть над судьбой."},
    {"album": "Donda", "year": 2021, "track": "Moon", "main_theme": "emotion", "subthemes": "escape", "commentary": "Поиск покоя в другом мире."},
    {"album": "Donda", "year": 2021, "track": "Heaven and Hell", "main_theme": "religion", "subthemes": "judgment", "commentary": "Дихотомия добра и зла."},
    {"album": "Donda", "year": 2021, "track": "Donda", "main_theme": "family", "subthemes": "mother", "commentary": "Память о матери, связь поколений."},
    {"album": "Donda", "year": 2021, "track": "Keep My Spirit Alive", "main_theme": "faith", "subthemes": "struggle", "commentary": "Духовная стойкость в трудностях."},
    {"album": "Donda", "year": 2021, "track": "Jesus Lord", "main_theme": "religion", "subthemes": "suffering, redemption", "commentary": "Один из самых серьёзных повествований о боли и спасении."},
    {"album": "Donda", "year": 2021, "track": "New Again", "main_theme": "forgiveness", "subthemes": "rebirth", "commentary": "Очистительное обновление после грехов."},
    {"album": "Donda", "year": 2021, "track": "Tell the Vision", "main_theme": "tribute", "subthemes": "Pop Smoke", "commentary": "Посвящение покойному Pop Smoke."},
    {"album": "Donda", "year": 2021, "track": "Lord I Need You", "main_theme": "relationships", "subthemes": "family, forgiveness", "commentary": "Молитва о сохранении семьи, посвящение Ким."},
    {"album": "Donda", "year": 2021, "track": "Pure Souls", "main_theme": "healing", "subthemes": "optimism", "commentary": "Трек о духовной лёгкости и новом начале."},
    {"album": "Donda", "year": 2021, "track": "Come to Life", "main_theme": "redemption", "subthemes": "emotion, transcendence", "commentary": "Оживление души, центральный момент альбома."},
    {"album": "Donda", "year": 2021, "track": "No Child Left Behind", "main_theme": "faith", "subthemes": "spiritual message", "commentary": "Утверждение Божьей поддержки."},

    # === DONDA 2 (2022) — часть ===
    {"album": "Donda 2", "year": 2022, "track": "True Love", "main_theme": "family", "subthemes": "divorce, children", "commentary": "Боль расставания и отношения с детьми."},
    {"album": "Donda 2", "year": 2022, "track": "Broken Road", "main_theme": "emotion", "subthemes": "reflection", "commentary": "Тёмные размышления о судьбе."},
    {"album": "Donda 2", "year": 2022, "track": "Get Lost", "main_theme": "introspection", "subthemes": "emptiness", "commentary": "Этюд о потерянной личности."},
    {"album": "Donda 2", "year": 2022, "track": "Too Easy", "main_theme": "ego", "subthemes": "success", "commentary": "Шоу-оф и самоутверждение."},
    {"album": "Donda 2", "year": 2022, "track": "Security", "main_theme": "anger", "subthemes": "paranoia, conflict", "commentary": "Паранойя и защита личных границ."},
    {"album": "Donda 2", "year": 2022, "track": "We Did It Kid", "main_theme": "celebration", "subthemes": "legacy", "commentary": "Победа, успех, достижение мечты."},
    {"album": "Donda 2", "year": 2022, "track": "Pablo", "main_theme": "identity", "subthemes": "chaos", "commentary": "Вихрь образов, нарциссизм."},
    {"album": "Donda 2", "year": 2022, "track": "Broken Road", "main_theme": "emotion", "subthemes": "reflection", "commentary": "Путь боли и исцеления."},
    {"album": "Donda 2", "year": 2022, "track": "I'm Finna Love Me", "main_theme": "self-love", "subthemes": "healing", "commentary": "О любви к себе и независимости."},
    {"album": "Donda 2", "year": 2022, "track": "Sci-Fi", "main_theme": "chaos", "subthemes": "divorce, surrealism", "commentary": "Разрыв отношений через футуристический образ."},
]

In [28]:
df = pd.DataFrame(tracks_data)

# Сохраняем в CSV
output_path = "kanye_tracks_themes_full.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print("CSV-файл успешно создан:", output_path)
print("Количество треков:", len(df))
print(df.head())

CSV-файл успешно создан: kanye_tracks_themes_full.csv
Количество треков: 222
                 album  year           track main_theme  \
0  The College Dropout  2004           Intro     social   
1  The College Dropout  2004   We Don't Care     social   
2  The College Dropout  2004  Graduation Day     social   
3  The College Dropout  2004  All Falls Down   identity   
4  The College Dropout  2004   I'll Fly Away   religion   

                    subthemes  \
0     education, expectations   
1  youth, poverty, inequality   
2      discipline, conformity   
3      insecurity, capitalism   
4                     freedom   

                                          commentary  
0  Сатира на давление общества и академических норм.  
1  Критика городской бедности и отсутствия возмож...  
2         Призыв не подчиняться системе образования.  
3  О низкой самооценке и потребительских стандартах.  
4     Госпел-вставка, символ духовного освобождения.  
